### aim: 

### date: 

In [149]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [150]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [151]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

In [152]:
import xarray as xr

changes_str = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/oceanFields/extracted-summary/all-medusa-50yrtrends.nc'
changes = xr.open_dataset(changes_str)
input_str = xr.open_dataset('./data/BOE-inputs/BOE-INPUT-CLQtest.nc')

ot_delta = changes.trend.loc[{'period':'1950-2000', 'season':'FY', 'quantity': 'ot', \
                      'forcing': 'ozone', 'scenario': 'ssp1' }].values *5 # / decade to / 50 yrs 

SO_area = 4.67e13


grad_DIC_surfdeep =\
input_str['grad_DIC_surfdeep'].loc[{'period':'1950-2000', 'season':'FY',\
                                    'scenario': 'ssp1' }].values 
mn_DIC =\
input_str['mn_DIC'].loc[{'period':'1950-2000', 'season':'FY',\
                                    'scenario': 'ssp1' }].values 
mn_pCO2 =\
input_str['mn_pCO2'].loc[{'period':'1950-2000', 'season':'FY',\
                                    'scenario': 'ssp1' }].values 
mn_Revfact =\
input_str['mn_Revfact'].loc[{'period':'1950-2000', 'season':'FY',\
                                    'scenario': 'ssp1' }].values 
mn_kg =\
input_str['mn_kg'].loc[{'period':'1950-2000', 'season':'FY',\
                                    'scenario': 'ssp1' }].values 
grad_dDICdz =\
input_str['grad_dDICdz'].loc[{'period':'1950-2000', 'season':'FY',\
                                    'scenario': 'ssp1' }].values 


print(f'gradient DIC surf/deep {grad_DIC_surfdeep}, mean surf dic {mn_DIC}, mean pco2 {mn_pCO2}, mean revfact {mn_Revfact}, kg {mn_kg}')


print('')
print('simple way of calculating OT:')

def effect_OT(ot_delta,grad_DIC_surfdeep, mn_Revfact, mn_DIC):

    dfco2_pgCyr_OT_DIC = ot_delta * 1e6 * grad_DIC_surfdeep * 1000*31560000*0.000001*0.000000000000012 #dfCO2_DIC = dOverturning x (dDICv)
    
    SO_area = 4.67e13
    SO_areaf = format(SO_area, '.2e')
    
    siy = 60*60*24*365 #seconds in year
    siyf = format(siy, '.2e')

    m3extra = ot_delta * 1e6 #extra m3 per second 
    m3extraf = format(m3extra, '.2e')
    
    instdicextra = m3extra * grad_DIC_surfdeep ##mmol/m3
    instdicextraf = format(instdicextra, '.2e')
 
    instdicextraconc = instdicextra/SO_area
    instdicextraconcf = format(instdicextraconc, '.2e')
    
    m3extrayear = m3extra * siy
    m3extrayearf = format(m3extrayear, '.2e')
    
    
    
    molextrayear =  m3extrayear * grad_DIC_surfdeep/1000
    molextrayearf = format(molextrayear, '.2e')
    
    pgextrayear = m3extrayear * grad_DIC_surfdeep/1000 * 1.2e-14
    pgextrayearf= format(pgextrayear, '.2e')
    

    
    print(f'OT change: {ot_delta} (say, between 2 years (unrealistic))')
    print(f'{ot_delta} extra SV means {ot_delta * 1e6} [{m3extraf}] m3/s extra deep water')
    print(f'times {60*60*24*365} [{siyf}] s/year')
    
    print(f'for a total of {m3extrayear} [{m3extrayearf}] m3 of extra deep water at surface')
    print(f'for each m3 of extra deep water, there are {grad_DIC_surfdeep} mmol, or {grad_DIC_surfdeep/1000} mol, of extra DIC')
    print(f'there are 1e15g in a pG, or 1e-15 pG in a gram, 12 g to a mol of carbon, so each extra mol is 1.2e-14 pG of carbon')
    print(f'that means there are {molextrayearf} extra mols, or {pgextrayearf} pG C delivered to the surface over the course of a year')
    print('')
    print(f'if we dont do a Revelle-factor style calculation, we can just assume that goes to flux')
    print('')
    print(f'if we do, we need to spread out these extra mols across the whole surface')
    print(f'and get a change in DIC concentration in mmol/m3')
    print(f'if we do, we need to know how big the surface ocean is: {SO_area} [{SO_areaf}] m2')
    print(f'at any given second in the year, there are {m3extraf} extra m3 deep water at the surface')
    print(f'bringing with them {instdicextraf} mmol DIC ({grad_DIC_surfdeep} extra mmol/m3)')
    print(f'which works out to an extra {instdicextraconcf} mmol/m3 DIC when dividing by surf oc area [{SO_areaf}] m2')
    print('')
    dpCO2 = np.max(mn_Revfact) * np.max(mn_pCO2) / np.max(mn_DIC) * instdicextraconc #dpCO2 = R x pCO2/DIC x dDIC, ppm
    
    print(f'we expect very little pco2 changes due to that amount (R * pco2/DIC * deltaDIC)')
    print(f'{dpCO2} ppm is pco2 change')
    dfCO2_DIC = mn_kg * dpCO2 #dfCO2_DIC = kg x dpCO2
    dfco2_pgCyr_DIC = dfCO2_DIC * 1.2e-14 * SO_area 
    print(f'which gives an extra carbon flux of:')
    print(f'{dfco2_pgCyr_DIC}Pg when multiplying by kg, the SO area, and converting to Pg Carbon')
#     print(f'for each m3 of extra deep water, there are {grad_DIC_surfdeep} mmol, or {grad_DIC_surfdeep/1000} mol, of extra DIC')
#     print(f'over the course of a year, that works out to {ot_delta*1e6*siy} extra mol')
#     print(f'there are 1e15g in a pG, or 1e-15 pG in a gram, 12 g to a mol of carbon')
#     print(f)
    
    return dfco2_pgCyr_OT_DIC #dfco2_pgCyr_OT_TA, dfco2_pgCyr_OT_Fe

dfco2_pgCyr_OT_DIC = effect_OT(ot_delta,grad_DIC_surfdeep, mn_Revfact, mn_DIC)

gradient DIC surf/deep 120.0, mean surf dic 2120.0, mean pco2 380.0, mean revfact 14.0, kg 0.065

simple way of calculating OT:
OT change: 4.753478020974628 (say, between 2 years (unrealistic))
4.753478020974628 extra SV means 4753478.020974628 [4.75e+06] m3/s extra deep water
times 31536000 [3.15e+07] s/year
for a total of 149905682869455.84 [1.50e+14] m3 of extra deep water at surface
for each m3 of extra deep water, there are 120.0 mmol, or 0.12 mol, of extra DIC
there are 1e15g in a pG, or 1e-15 pG in a gram, 12 g to a mol of carbon, so each extra mol is 1.2e-14 pG of carbon
that means there are 1.80e+13 extra mols, or 2.16e-01 pG C delivered to the surface over the course of a year

if we dont do a Revelle-factor style calculation, we can just assume that goes to flux

if we do, we need to spread out these extra mols across the whole surface
and get a change in DIC concentration in mmol/m3
if we do, we need to know how big the surface ocean is: 46700000000000.0 [4.67e+13] m2
at an

### sst carbon flux calculation

https://oceanobservatories.org/wp-content/uploads/2023/09/1341-00270_Data_Product_SPEC_CO2FLUX_OOI.pdf

wannikhof 1992: Gas transfer of CO2 is sometimes expressed as a  gas transfer coefficient K [Etcheto and Merlivat, 1988; Tans et al., 1990; Thomas et al., 1988]. The flux equals the gas transfer coefficient multiplied by the partial pressure differ- ence between air and water: F  =  K(pCO2w -  pCO2a) (A2) where K - kL and L is the solubility expressed in units of (concentration/pressure). For CO2 the increase in Sc -1/2 (and thus k) with temperature is nearly compensated by a decrease in L such that K is nearly temperature indepen- dent. For example, (Sc-•/2L)sc =  1.07 (Sc-•/2L)25c ,, while the Schmidt number changes by a factor of 3 over this range. In most recent estimates of CO2 fluxes between the ocean and the atmosphere, K is assumed to be independent of temperature. Although the approximation will not introduce large errors, especially in light of other uncertainties deter- mining K, it  is preferable to use the exact form, since the approximation will cause systematic bias with temperature. This bias will be pronounced because the warmer ocean waters are frequently CO2 sources, while the colder waters are CO2 sinks.

In [190]:
t = 2; S = 35.0; pCO2W = 390.0; pCO2A = 380.0; u10 = 10.0


def co2flux(t,S,pCO2W,pCO2A,u10):

    pCO2W = pCO2W/1e6
    pCO2A = pCO2A/1e6
    #schmidt
    Sc = 2073.1 - 125.62*t + 3.6276*t*t - 0.043219*t*t*t

    #gas transfer
    k = (0.27*u10*u10 * np.sqrt(660/Sc))/360000

    absT = t + 273.15

    #solubility (in mol atm-1 m-3)
    Ko = 1000 * np.exp( -58.0931 + 90.5069*(100/absT) + 22.2940 * np.log(absT/100))\
    + S*(0.027766 - 0.025888* (absT/100) + 0.0050578* (absT/100) * (absT/100))

    F = k * Ko * (pCO2W - pCO2A) #mol m2/s
    
    print(f'temp{t}degC, sal{S}, pCO2W{pCO2W*1e6}uatm, pCO2A{pCO2A*1e6}uatm, windspeed {u10}m/s')
    print(f'flux is {F} mol/m2/s')
    
    SO_area = 4.67e13
    siy = 60*60*24*365 #seconds in year
    
    FPg = F * SO_area * siy * 1.2e-14 #PgC/mol
    print(f'or {FPg} Pg/year taken over whole SO')
    print()
    return F, FPg

t = 2; S = 35.0; pCO2A = 700.; pCO2W = pCO2A-45.42; u10 = 10.0

# print()
# F = co2flux(t,S,pCO2W,pCO2A,u10)
# F = co2flux(t+1,S,pCO2W,pCO2A,u10)


F, FPG = co2flux(t,S,pCO2W,pCO2A,u10)
pco2w_2 = pCO2W*0.04+pCO2W
F1, FPG1 = co2flux(t+1,S,pco2w_2,pCO2A,u10)

print(f'at 2002 mean ocean pco2 values ({pCO2W})uatm and atm pco2 values ({pCO2A})')
print(f'the co2 sink is {F} mol/m2/s or {FPG} Pg/yr over the whole SO')
print(f'an increase of 1 degree T leads to {pco2w_2}, or {pco2w_2-pCO2W}uatm extra')
print(f'which would mean a co2 sink of {F1} mol/m2/s or {FPG1} Pg/yr over the whole SO')
print(f'so the co2 sink has reduced by {FPG-FPG1} pG/year')

temp2degC, sal35.0, pCO2W654.58uatm, pCO2A700.0uatm, windspeed 10.0m/s
flux is -1.4615976277860202e-07 mol/m2/s
or -2.583048513943751 Pg/year taken over whole SO

temp3degC, sal35.0, pCO2W680.7632000000001uatm, pCO2A700.0uatm, windspeed 10.0m/s
flux is -6.14145529032646e-08 mol/m2/s
or -1.0853655383362604 Pg/year taken over whole SO

at 2002 mean ocean pco2 values (654.58)uatm and atm pco2 values (700.0)
the co2 sink is -1.4615976277860202e-07 mol/m2/s or -2.583048513943751 Pg/yr over the whole SO
an increase of 1 degree T leads to 680.7632000000001, or 26.183200000000056uatm extra
which would mean a co2 sink of -6.14145529032646e-08 mol/m2/s or -1.0853655383362604 Pg/yr over the whole SO
so the co2 sink has reduced by -1.4976829756074905 pG/year


## ok mocsy

In [154]:
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/mocsy-master')
import mocsy
import gsw

In [172]:
DIC = 2100e-6
ALK = 2300e-6
votemper = 2
vosaline = 35

def mocsy_wrap(DIC, ALK, votemper, vosaline):

    tdra = np.ravel(DIC * 1e3) #DIC, convert mol/L to mol/m3 (1000 L/m3)
    ttara = np.ravel(ALK * 1e3) #Alkalinity, convert mol/L to mol/m3 (1000 L/m3)
    tsra = np.ravel(vosaline) #salt
    tsra_psu = np.ravel(vosaline) *35/35.16504 #g/kg to psu (salt)
    ttera = np.ravel(votemper)

    tdepth = np.zeros_like(ttera)
    tdepth[:] = 0
    ttera_is = gsw.t_from_CT(tsra,ttera,tdepth)
    tzero = np.zeros_like(tdepth)
    tpressure = np.zeros_like(tdepth)

    response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=ttara, dic=tdra, 
                       sil=tzero, phos=tzero, patm=tpressure, depth=tdepth, lat=tzero, 
                        optcon='mol/m3', optt='Tinsitu', optp='m',
                        optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    pco2 = (pco2[0])
    
    print(f'at DIC of {DIC*1e6} umol/L, TA of {ALK*1e6} umol/L, t of {votemper}C, S of {vosaline} psu:')
    print(f'pco2 is {pco2}')
    print()
    
    return pco2

DIC = 2100e-6
ALK = 2300e-6
votemper = 2
vosaline = 35
pco2 = mocsy_wrap(DIC, ALK, votemper, vosaline)

pco2_1 = mocsy_wrap(DIC, ALK, votemper+1, vosaline)
print(f'{(pco2_1-pco2)/pco2 * 100} percent increase with deg T')
print('----')

DIC = 2150e-6
ALK = 2300e-6
votemper = 2
vosaline = 35
pco2 = mocsy_wrap(DIC, ALK, votemper, vosaline)

pco2_1 = mocsy_wrap(DIC, ALK, votemper+1, vosaline)
print(f'{(pco2_1-pco2)/pco2 * 100} percent increase with deg T')
print('----')

DIC = 2250e-6
ALK = 2300e-6
votemper = 2
vosaline = 35
pco2 = mocsy_wrap(DIC, ALK, votemper, vosaline)

pco2_1 = mocsy_wrap(DIC, ALK, votemper+1, vosaline)
print(f'{(pco2_1-pco2)/pco2 * 100} percent increase with deg T')
print('----')

at DIC of 2100.0 umol/L, TA of 2300.0 umol/L, t of 2C, S of 35 psu:
pco2 is 254.99431579230264

at DIC of 2100.0 umol/L, TA of 2300.0 umol/L, t of 3C, S of 35 psu:
pco2 is 266.6839513707269

4.584273003146342 percent increase with deg T
----
at DIC of 2150.0 umol/L, TA of 2300.0 umol/L, t of 2C, S of 35 psu:
pco2 is 346.57473177586184

at DIC of 2150.0 umol/L, TA of 2300.0 umol/L, t of 3C, S of 35 psu:
pco2 is 362.3152727672469

4.541745126867712 percent increase with deg T
----
at DIC of 2250.0 umol/L, TA of 2300.0 umol/L, t of 2C, S of 35 psu:
pco2 is 719.7266737726288

at DIC of 2250.0 umol/L, TA of 2300.0 umol/L, t of 3C, S of 35 psu:
pco2 is 751.1229696504408

4.362252647000063 percent increase with deg T
----
